# 01 — Exploratory Data Analysis

Load the processed monthly panel and explore the data.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style("whitegrid")
pd.options.display.max_columns = 20

In [ ]:
panel = pd.read_parquet("../data/processed/panel.parquet")
print(f"Shape: {panel.shape}")
print(f"Range: {panel.index.min().date()} to {panel.index.max().date()}")
panel.head(10)

## Missing Values

In [ ]:
panel.isnull().sum()

## Time Series Plots

In [ ]:
fig, axes = plt.subplots(len(panel.columns), 1, figsize=(14, 3 * len(panel.columns)), sharex=True)
for ax, col in zip(axes, panel.columns):
    ax.plot(panel.index, panel[col])
    ax.set_ylabel(col, fontsize=8)
    ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## Correlation Matrix

In [ ]:
fig, ax = plt.subplots(figsize=(10, 8))
sns.heatmap(panel.corr(), annot=True, fmt=".2f", cmap="RdBu_r", center=0, ax=ax)
ax.set_title("Correlation Matrix")
plt.tight_layout()
plt.show()

## Descriptive Statistics

In [ ]:
panel.describe()